<a href="https://colab.research.google.com/github/Rachhh53/MSDS-460-Decision-Analytics/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

     |████████████████████████████████| 14.2 MB 4.6 MB/s 


In [ ]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [2]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK
import pandas as pd

# Problem 1

1a.

In [ ]:
# define variables
x1 = LpVariable("x1", None)
x2 = LpVariable("x2", None, 0, None)
x3 = LpVariable("x3", 0, None)
x4 = LpVariable("x4", 0, None)

In [ ]:
# defines the problem
prob1a = LpProblem("problem", LpMaximize)

In [ ]:
# define constraints
prob1a += 4*x1 + x2 + x3 == 20
prob1a += 2*x1 - x2 >= 6
prob1a += x1 - x2 + 5*x3 >= -5
prob1a += -3*x1 + 2*x2 + x3 <= 4

In [ ]:
# define objective function
prob1a += -4*x1 + 2*x2

In [ ]:
prob1a.solve()
print(LpStatus[prob1a.status])
for i in prob1a.variables():
    print("Variable {0} = {1}".format(i.name, i.varValue))
print("Objective function z = {0}".format((prob1a.objective)))

Optimal
Variable x1 = 2.0
Variable x2 = -2.0
Variable x3 = 14.0
Objective function z = -4*x1 + 2*x2


In [ ]:
for name, c in list(prob1a.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)

_C1 : 4*x1 + x2 + x3 = 20 	 -0.0 		 -0.0
_C2 : 2*x1 - x2 >= 6 	 -2.0 		 -0.0
_C3 : x1 - x2 + 5*x3 >= -5 	 -0.0 		 -79.0
_C4 : -3*x1 + 2*x2 + x3 <= 4 	 -0.0 		 -0.0


Looks like there is not a good way to systematically get the dual...

In [ ]:

# define variables
x1 = LpVariable("x1", 0, None)
x2 = LpVariable("x2", 0, None)
x3 = LpVariable("x3", 0, None)

In [ ]:
# defines the problem
prob1b = LpProblem("problem", LpMaximize)

In [ ]:
# define constraints
prob1b += 8*x1 + 6*x2 + x3 <= 50
prob1b += 4*x1 + 2*x2 + 3*x3 <= 20
prob1b += 2*x1 + x2 + 2*x3 <= 25

In [ ]:
# define objective function
prob1b += 25*x1 + 30*x2 + 20*x3

In [ ]:
# solve the problem
prob1b.writeLP("prob1b.lp")
prob1b.solve(GLPK(options=['--ranges', 'prob1b.sen']))
print ("Status:", LpStatus[prob1b.status])

for v in prob1b.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob1b.objective))
print ("")

Status: Optimal
x1 = 0.0
x2 = 8.125
x3 = 1.25
Objective 268.75



# Problem 2

In [49]:
# define variables
#M1 = LpVariable("M1", 0, 98) # number of type 1 machines available
#M2 = LpVariable("M2", 0, 73) # number of type 2 machines available
#S = LpVariable("S", 0, 260) # tons of steel available
TM1 = LpVariable("TM1", 0, None) # trucks produced on type 1 machine
TM2 = LpVariable("TM2", 0, None) # trucks produced on type 2 machine
CM1 = LpVariable("CM1", 0, None) # cars produced on type 1 machine
CM2 = LpVariable("CM2", 0, None) # cars produced on type 2 machine

In [50]:
# defines the problem
prob2 = LpProblem("problem", LpMaximize)

In [51]:
# define constraints
prob2 += CM1 + CM2 >= 88 # Marketing considerations dictate that at least 88 cars must be sold
prob2 += TM1 + TM2 >= 26 # Marketing considerations dictate that at least 26 trucks must be sold
#prob2 += TM1 + CM1 == M1 # cars and trucks produced on machine 1 need to equal the total of machine 1
#prob2 += TM2 + CM2 == M2 # cars and trucks produced on machine 2 need to equal the total of machine 2
prob2 += 2*(CM1 + CM2) + 3*(TM1 + TM2) <= 260 # steel used is less than or equal to available
prob2 += .8*CM1 + TM1 <= 98  # cars and trucks time to produce on machine 1 must be <= machines available
prob2 += .6*CM2 + .7*TM2 <= 73 # cars and trucks time to produce on machine 2 must be <= machines available

In [52]:
# define objective function
prob2 += 300*(CM1 + CM2) + 400*(TM1 + TM2) - 50*(CM1 + TM1)

In [53]:
# solve the problem
prob2.writeLP("prob2.lp")
prob2.solve(GLPK(options=['--ranges', 'prob2.sen']))
print ("Status:", LpStatus[prob2.status])

for v in prob2.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob2.objective))
print ("")

Status: Optimal
CM1 = 0.0
CM2 = 91.0
TM1 = 0.0
TM2 = 26.0
Objective 37700.0



useless?

In [54]:
print(f"dual:\n")
o = [{'name':name, 'constraint': c, 'shadow price':c.pi, 'slack': c.slack} 
     for name, c in prob2.constraints.items()]
print(pd.DataFrame(o))

dual:

  name                        constraint shadow price slack
0  _C1                  {CM1: 1, CM2: 1}         None  None
1  _C2                  {TM1: 1, TM2: 1}         None  None
2  _C3  {CM1: 2, CM2: 2, TM1: 3, TM2: 3}         None  None
3  _C4                {CM1: 0.8, TM1: 1}         None  None
4  _C5              {CM2: 0.6, TM2: 0.7}         None  None


2i - just confirming!

In [ ]:
# define variables
#M1 = LpVariable("M1", 0, 98) # number of type 1 machines available
#M2 = LpVariable("M2", 0, 73) # number of type 2 machines available
#S = LpVariable("S", 0, 260) # tons of steel available
TM1 = LpVariable("TM1", 0, None) # trucks produced on type 1 machine
TM2 = LpVariable("TM2", 0, None) # trucks produced on type 2 machine
CM1 = LpVariable("CM1", 0, None) # cars produced on type 1 machine
CM2 = LpVariable("CM2", 0, None) # cars produced on type 2 machine

In [ ]:
# defines the problem
prob2 = LpProblem("problem", LpMaximize)

In [ ]:
# define constraints
prob2 += CM1 + CM2 >= 88 # Marketing considerations dictate that at least 88 cars must be sold
prob2 += TM1 + TM2 >= 26 # Marketing considerations dictate that at least 26 trucks must be sold
#prob2 += TM1 + CM1 == M1 # cars and trucks produced on machine 1 need to equal the total of machine 1
#prob2 += TM2 + CM2 == M2 # cars and trucks produced on machine 2 need to equal the total of machine 2
prob2 += 2*(CM1 + CM2) + 3*(TM1 + TM2) <= 260 # steel used is less than or equal to available
prob2 += .8*CM1 + TM1 <= 98  # cars and trucks time to produce on machine 1 must be <= machines available
prob2 += .6*CM2 + .7*TM2 <= 73 # cars and trucks time to produce on machine 2 must be <= machines available

In [35]:
# define objective function
prob2 += 310*(CM1 + CM2) + 400*(TM1 + TM2) - 50*(CM1 + TM1)

In [36]:
# solve the problem
prob2.writeLP("prob2.lp")
prob2.solve(GLPK(options=['--ranges', 'prob2.sen']))
print ("Status:", LpStatus[prob2.status])

for v in prob2.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob2.objective))
print ("")

Status: Optimal
CM1 = 0.0
CM2 = 91.0
TM1 = 0.0
TM2 = 26.0
Objective 38610.0



2iv - check

In [37]:
# define variables
#M1 = LpVariable("M1", 0, 98) # number of type 1 machines available
#M2 = LpVariable("M2", 0, 73) # number of type 2 machines available
#S = LpVariable("S", 0, 260) # tons of steel available
TM1 = LpVariable("TM1", 0, None) # trucks produced on type 1 machine
TM2 = LpVariable("TM2", 0, None) # trucks produced on type 2 machine
CM1 = LpVariable("CM1", 0, None) # cars produced on type 1 machine
CM2 = LpVariable("CM2", 0, None) # cars produced on type 2 machine

In [38]:
# defines the problem
prob2 = LpProblem("problem", LpMaximize)

In [39]:
# define constraints
prob2 += CM1 + CM2 >= 86 # Marketing considerations dictate that at least 88 cars must be sold
prob2 += TM1 + TM2 >= 26 # Marketing considerations dictate that at least 26 trucks must be sold
#prob2 += TM1 + CM1 == M1 # cars and trucks produced on machine 1 need to equal the total of machine 1
#prob2 += TM2 + CM2 == M2 # cars and trucks produced on machine 2 need to equal the total of machine 2
prob2 += 2*(CM1 + CM2) + 3*(TM1 + TM2) <= 260 # steel used is less than or equal to available
prob2 += .8*CM1 + TM1 <= 98  # cars and trucks time to produce on machine 1 must be <= machines available
prob2 += .6*CM2 + .7*TM2 <= 73 # cars and trucks time to produce on machine 2 must be <= machines available

In [40]:
# define objective function
prob2 += 310*(CM1 + CM2) + 400*(TM1 + TM2) - 50*(CM1 + TM1)

In [41]:
# solve the problem
prob2.writeLP("prob2.lp")
prob2.solve(GLPK(options=['--ranges', 'prob2.sen']))
print ("Status:", LpStatus[prob2.status])

for v in prob2.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob2.objective))
print ("")

Status: Optimal
CM1 = 0.0
CM2 = 91.0
TM1 = 0.0
TM2 = 26.0
Objective 38610.0



2.v check

In [42]:
# define variables
#M1 = LpVariable("M1", 0, 98) # number of type 1 machines available
#M2 = LpVariable("M2", 0, 73) # number of type 2 machines available
#S = LpVariable("S", 0, 260) # tons of steel available
TM1 = LpVariable("TM1", 0, None) # trucks produced on type 1 machine
TM2 = LpVariable("TM2", 0, None) # trucks produced on type 2 machine
CM1 = LpVariable("CM1", 0, None) # cars produced on type 1 machine
CM2 = LpVariable("CM2", 0, None) # cars produced on type 2 machine
JM1 = LpVariable("JM1", 0, None) # cars produced on type 2 machine
JM2 = LpVariable("JM2", 0, None) # cars produced on type 2 machine

In [43]:
# defines the problem
prob2 = LpProblem("problem", LpMaximize)

In [45]:
# define constraints
prob2 += CM1 + CM2 >= 88 # Marketing considerations dictate that at least 88 cars must be sold
prob2 += TM1 + TM2 >= 26 # Marketing considerations dictate that at least 26 trucks must be sold
#prob2 += TM1 + CM1 == M1 # cars and trucks produced on machine 1 need to equal the total of machine 1
#prob2 += TM2 + CM2 == M2 # cars and trucks produced on machine 2 need to equal the total of machine 2
prob2 += 2*(CM1 + CM2) + 3*(TM1 + TM2) +4* (JM1+JM2) <= 260 # steel used is less than or equal to available
prob2 += .8*CM1 + TM1 + 1.2*JM1 <= 98  # cars and trucks time to produce on machine 1 must be <= machines available
prob2 += .6*CM2 + .7*TM2 + 2*JM2 <= 73 # cars and trucks time to produce on machine 2 must be <= machines available

In [46]:
# define objective function
prob2 += 310*(CM1 + CM2) + 400*(TM1 + TM2) + 600*(JM1 + JM2) - 50*(CM1 + TM1)

In [47]:
# solve the problem
prob2.writeLP("prob2.lp")
prob2.solve(GLPK(options=['--ranges', 'prob2.sen']))
print ("Status:", LpStatus[prob2.status])

for v in prob2.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob2.objective))
print ("")

Status: Optimal
CM1 = 0.0
CM2 = 91.0
JM1 = 0.0
JM2 = 0.0
TM1 = 0.0
TM2 = 26.0
Objective 38610.0



# Problem 3

A catering company must have the following number of clean napkins available at the beginning of each of the next four days:<br>
* day 1: 15
* day 2: 12
* day 3: 18
* day 4: 6 




In [8]:
# define variables
d1s = LpVariable("day1-slow", 0, None)
d1f = LpVariable("day1-fast", 0, None)
d1p = LpVariable("day1-purchased", 15, 15) # must purchase enough for the first day, but why buy more?
d2s = LpVariable("day2-slow", 0, None)
d2f = LpVariable("day2-fast", 0, None)
d2p = LpVariable("day2-purchased", 0, None)
d3f = LpVariable("day3-fast", 0, 18) # won't clean more than 
d3p = LpVariable("day3-purchased", 0, None)
d4p = LpVariable("day4-purchased", 0, 6) # won't purchase more than is needed for the last day

# day 3 slow doesn't exisit because the world of this problem ends on day 4
# so better wash all them napkins

In [9]:
# define the problem
prob3 = LpProblem("problem", LpMinimize)

In [10]:
# define constraints
# day 1
prob3 += d1f + d1s <= d1p # max to clean is what you bought/used
# day 2
prob3 += d1f + d2p >= 12 # puchase what you didn't clean fast from day 1
prob3 += d2f + d2s <= d2p + d1f # can only clean what is dirty
# day 3
prob3 += d2f + d1s + d3p >= 18 # day 2 fast, day 1 slow and purchsed on day 3 available for day 3
prob3 += d3f <= d2f + d1s + d3p # there is only one more day so no slow wash this day
# day 4
prob3 += d3f + d4p + d2s >= 6 # day 3 fast, day 4 purchases, day 2 slow all available for day 4

In [11]:
# define objective function
prob3 += .06*(d1s + d2s) + .1*(d1f + d2f + d3f) + .2*(d1p + d2p + d3p + d4p)

In [13]:
# solve the problem
status = prob3.solve()
print(f"Problem")
print(f"status={LpStatus[status]}")

# print the results
for variable in prob3.variables():
    print(f"{variable.name} = {variable.varValue}")

print(f"Objective = {value(prob3.objective)}")
print(f"")

Problem
status=Optimal
day1_fast = 9.0
day1_purchased = 15.0
day1_slow = 6.0
day2_fast = 12.0
day2_purchased = 3.0
day2_slow = 0.0
day3_fast = 6.0
day3_purchased = 0.0
day4_purchased = 0.0
Objective = 6.66



# Problem 4

Requirements:<br>
* Four sections of marketing, finance and production must be offered each year
* At least one section of each class must be offered during both Spring and Fall semesters



In [14]:
# define variables
# Marketing
fm1 = LpVariable("Fall-Marketing-Prof1", 0, None)
fm2 = LpVariable("Fall-Marketing-Prof2", 0, None)
fm3 = LpVariable("Fall-Marketing-Prof3", 0, None)
sm1 = LpVariable("Spring-Marketing-Prof1", 0, None)
sm2 = LpVariable("Spring-Marketing-Prof2", 0, None)
sm3 = LpVariable("Spring-Marketing-Prof3", 0, None)
# Finance
ff1 = LpVariable("Fall-Finance-Prof1", 0, None)
ff2 = LpVariable("Fall-Finance-Prof2", 0, None)
ff3 = LpVariable("Fall-Finance-Prof3", 0, None)
sf1 = LpVariable("Spring-Finance-Prof1", 0, None)
sf2 = LpVariable("Spring-Finance-Prof2", 0, None)
sf3 = LpVariable("Spring-Finance-Prof3", 0, None)
# Production (whatever that is)
fp1 = LpVariable("Fall-Production-Prof1", 0, None)
fp2 = LpVariable("Fall-Production-Prof2", 0, None)
fp3 = LpVariable("Fall-Production-Prof3", 0, None)
sp1 = LpVariable("Spring-Production-Prof1", 0, None)
sp2 = LpVariable("Spring-Production-Prof2", 0, None)
sp3 = LpVariable("Spring-Production-Prof3", 0, None)

In [15]:
prob4 = LpProblem("problem", LpMinimize)

In [16]:
# define constraints
# each professor must teach 4 classes per year
prob4 += fm1 + sm1 + ff1 + sf1 + fp1 + sp1 == 4 # professor 1
prob4 += fm2 + sm2 + ff2 + sf2 + fp2 + sp2 == 4 # professor 2
prob4 += fm3 + sm3 + ff3 + sf3 + fp3 + sp3 == 4 # professor 3

# there are 4 sections of each class per year
prob4 += fm1 + fm2 + fm3 + sm1 + sm2 + sm3 == 4 # marketing
prob4 += ff1 + ff2 + ff3 + sf1 + sf2 + sf3 == 4 # finance
prob4 += fp1 + fp2 + fp3 + sp1 + sp2 + sp3 == 4 # production

# at least one sectino of each class must be offered during each semester
prob4 += fm1 + fm2 + fm3 >= 1 # fall marketing
prob4 += sm1 + sm2 + sm3 >= 1 # spring marketing
prob4 += ff1 + ff2 + ff3 >= 1 # fall finance
prob4 += sf1 + sf2 + sf3 >= 1 # spring finance
prob4 += fp1 + fp2 + fp3 >= 1 # fall production
prob4 += sp1 + sp2 + sp3 >= 1 # spring production

In [17]:
# define objective function
prob4 += (3+6)*fm1 + (4+6)*sm1 + (3+5)*ff1 + (4+5)*sf1 + (3+4)*fp1 + (4+4)*sp1 \
+ (5+4)*fm2 + (3+4)*sm2 + (5+6)*ff2 + (3+6)*sf2 + (5+5)*fp2 + (3+5)*sp2 \
+ (4+5)*fm3 + (4+5)*sm3 + (4+4)*ff3 + (4+4)*sf3 + (4+6)*fp3 + (4+6)*sp3

In [19]:
# solve the problem
status = prob4.solve()
print(f"Problem")
print(f"status={LpStatus[status]}")

# print the results
for variable in prob4.variables():
    print(f"{variable.name} = {variable.varValue}")

print(f"Objective = {value(prob4.objective)}")
print(f"")

Problem
status=Optimal
Fall_Finance_Prof1 = 0.0
Fall_Finance_Prof2 = 0.0
Fall_Finance_Prof3 = 3.0
Fall_Marketing_Prof1 = 1.0
Fall_Marketing_Prof2 = 0.0
Fall_Marketing_Prof3 = 0.0
Fall_Production_Prof1 = 3.0
Fall_Production_Prof2 = 0.0
Fall_Production_Prof3 = 0.0
Spring_Finance_Prof1 = 0.0
Spring_Finance_Prof2 = 0.0
Spring_Finance_Prof3 = 1.0
Spring_Marketing_Prof1 = 0.0
Spring_Marketing_Prof2 = 3.0
Spring_Marketing_Prof3 = 0.0
Spring_Production_Prof1 = 0.0
Spring_Production_Prof2 = 1.0
Spring_Production_Prof3 = 0.0
Objective = 91.0

